In [2]:
import pandas as pd
from vega import VegaLite

In [5]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [9]:
VegaLite({
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "mark": "point",
  "encoding": {
    "y": {"type": "quantitative", "field": "sepal_length"},
    "x": {"type": "quantitative", "field": "sepal_width"}
  }
}, iris)